In [2]:
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
import google.generativeai as genai
from langdetect import detect
import requests
import json
import os
from dotenv import load_dotenv

load_dotenv()

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Configure Gemini
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel('gemini-2.0-flash')

# Google Safe Browsing API
GOOGLE_SAFE_BROWSING_API_KEY = os.getenv("GOOGLE_SAFE_BROWSING_API_KEY")

def analyze_text(text: str) -> dict:
    lang = detect(text)
    if lang != 'en':
        text = translate_text(text, src=lang, dest='en')
    
    prompt = f"""
    Analyze this message for scams. Check for urgency, financial requests, or impersonation.
    Output strict JSON: {{"risk_level": "Safe/Suspicious/Scam", "justification": "..."}}.
    Message: {text}
    """
    
    response = model.generate_content(prompt)
    result = json.loads(response.text)
    
    if lang != 'en':
        result['justification'] = translate_text(result['justification'], src='en', dest=lang)
    
    return result

def transcribe_voice(audio_file_bytes: bytes) -> str:
    audio_model = genai.GenerativeModel('gemini-2.0-flash')
    response = audio_model.generate_content(
        contents=[audio_file_bytes, "Transcribe this audio message"],
        request_options={"timeout": 10}
    )
    return response.text

@app.post("/scan/text")
async def scan_text(message: str = Form(...)):
    try:
        result = analyze_text(message)
        return result
    except Exception as e:
        return JSONResponse(status_code=500, content={"error": str(e)})

@app.post("/scan/voice")
async def scan_voice(file: UploadFile = File(...)):
    try:
        audio_bytes = await file.read()
        text = transcribe_voice(audio_bytes)
        result = analyze_text(text)
        return result
    except Exception as e:
        return JSONResponse(status_code=500, content={"error": str(e)})

# URL scanning remains same as previous implementation
@app.post("/scan/url")
async def scan_url(url: str = Form(...)):
    api_url = "https://safebrowsing.googleapis.com/v4/threatMatches:find"
    payload = {
        "threatInfo": {
            "threatTypes": ["MALWARE", "PHISHING"],
            "platformTypes": ["ANY"],
            "threatEntryTypes": ["URL"],
            "threatEntries": [{"url": url}]
        }
    }
    params = {"key": GOOGLE_SAFE_BROWSING_API_KEY}
    response = requests.post(api_url, params=params, json=payload)
    if response.status_code == 200:
        data = response.json()
        risk = "Scam" if data.get("matches") else "Safe"
        return {"url": url, "risk": risk, "details": data.get("matches", [])}
    else:
        return JSONResponse(status_code=500, content={"error": "Failed to check URL"})
